Sophia Walton, srw9rx

14 november 2021

# Homework 04 Language Modeling

## 0. Suggestions and Requirements

- Search the keyword TODO will help you quickly locate the questions in this assignment.
- Please **keep** all the outputs in your submission. Our TA is not required to run the code for grading. 
- Training the bug-free version of RNN LM (section 2) may need 30 minutes or longer, depending your local hardware. So when you implement/debug your code, you can create a small training file (e.g., only use the first 1K sentences) for the purpose of testing.


First, let's download the data for this homework.

In [1]:
# Download the data

import urllib.request
from os.path import isfile
if not isfile("arxiv.zip"):
    url = "https://yangfengji.net/uva-nlp-course/data/arxiv.zip"
    print("Downloading ...")
    filename, headers = urllib.request.urlretrieve(url, filename="arxiv.zip")

print("Decompressing the file ...")
!unzip arxiv.zip

sents = open("arxiv/trn-arxiv.txt").read().split("\n")
print("Read {} sentences from the training set".format(len(sents)))

Decompressing the file ...
Archive:  arxiv.zip
   creating: arxiv/
  inflating: arxiv/trn-arxiv.txt     
  inflating: arxiv/dev-arxiv.txt     
Read 63259 sentences from the training set


## 1. Statistical Language Modeling

The following homework is designed based on the demo code of statistical language modeling. In this homework, you will be asked to implemented two additional components besides the demo code

1. Parameter estimation with discounting
2. Perplexity calculation

Our implementation starts from the following demo code as we used in our class lecture, which will save us some time to implement the `logprob()` function and the `generate()` function.

In [2]:
from collections import defaultdict
from math import log2, pow
from numpy.random import choice

# This is the same model used in our class lecture demo code

class BigramLM(object):
    def __init__(self):
        self.vocab = {"<start>":0, "<end>":1}
        self.model = {}
        self.tok_counter = '__total__'

    def build(self, fname):
        """ Build a Bigram LM
        """
        fin = open(fname)
        for line in fin:
            tokens = line.strip().split()
            L = len(tokens)
            for i in range(1, L):
                prev_tok = tokens[i-1]
                curr_tok = tokens[i]
                if curr_tok not in self.vocab:
                    self.vocab.update({curr_tok : len(self.vocab)})
                try:
                    self.model[prev_tok][curr_tok] += 1.0
                except KeyError:
                    self.model[prev_tok] = defaultdict(float)
                    self.model[prev_tok][curr_tok] += 1.0
                self.model[prev_tok][self.tok_counter] += 1.0
        # Normalization
        for (prev_tok, dct) in self.model.items():
            for (curr_tok, val) in self.model[prev_tok].items():
                if curr_tok != self.tok_counter: # to avoid normalizing the counter token
                    self.model[prev_tok][curr_tok] /= self.model[prev_tok][self.tok_counter]
        print("Done with modeling building\nVocab size = {}".format(len(self.vocab)))
            

    def logprob(self, text):
        """ Evaluate a given text
        """
        tokens = text.strip().split()
        L = len(tokens)
        logprob = 0.0
        for i in range(1, L):
            prev_tok = tokens[i-1]
            curr_tok = tokens[i]
            if prev_tok not in self.vocab:
                prev_tok = 'UNK'
            if curr_tok not in self.vocab:
                curr_tok = 'UNK'
            try:
                logprob += log2(self.model[prev_tok][curr_tok])
            except ValueError:
                print("Missing word pair: {} -> {} from the probability table".format(prev_tok, curr_tok))
                logprob += -1000 # A large number, technically this should be infty
        return logprob


    def generate(self, method="random", length=20):
        """ Random sampling words from this model for generation
        """
        text = []
        prev_tok = "<start>"
        text.append(prev_tok)
        while (prev_tok != "<end>") and (len(text) <= length):
            tokens, probs = [], []
            # The following for loop is time-consuming
            # For large-scale text generation, a pre-processing may be necessary 
            for (tok, prob) in self.model[prev_tok].items():
                if tok != self.tok_counter:
                    tokens.append(tok)
                    probs.append(prob)
                
                #print(prev_tok, tokens[probs.index(max(probs))])
            widx = choice(len(probs), 1, p=probs)[0]
            prev_tok = tokens[widx]
            text.append(prev_tok)
        text = " ".join(text)
        return text            

### 1.1 Discounting (3 points)

$$P(x_t\mid x_{t-1}) = \frac{c(x_{t-1},x_t) + \alpha}{c(x_{t-1}) + V\alpha}$$
where $V$ is the vocabulary size and $\alpha$ is the hyper-parameter of discounting.

The following class `BigramLM_Discounted` is inherited from the class `BigramLM`, which is implemented in the demo code for our lecture. 

In the following code block, please implement the discounting technique as shown in the above equation. 

In [3]:
class BigramLM_Discounted(BigramLM):
    '''
    The logprob() function and the generate() function will be inherited from the BigramLM class. 
    The only thing left here is the build() function for building language models with discouting. 
    '''
    def build(self, fname, alpha=0.1):
        fin = open(fname)
        for line in fin:
            tokens = line.strip().split()
            L = len(tokens)
            for i in range(1, L):
                prev_tok = tokens[i-1]
                curr_tok = tokens[i]
                if curr_tok not in self.vocab:
                    self.vocab.update({curr_tok : len(self.vocab)})
                try:
                    self.model[prev_tok][curr_tok] += 1.0
                except KeyError:
                    self.model[prev_tok] = defaultdict(float)
                    self.model[prev_tok][curr_tok] += (1.0+alpha) #add 1+alpha to value for having alpha in each one
                    self.model[prev_tok][self.tok_counter]+=alpha #for each first item, add alpha value in
                self.model[prev_tok][self.tok_counter] += 1.0
        # Normalization
        for (prev_tok, dct) in self.model.items():
            for (curr_tok, val) in self.model[prev_tok].items():
                if curr_tok != self.tok_counter: # to avoid normalizing the counter token
                    self.model[prev_tok][curr_tok] /= (self.model[prev_tok][self.tok_counter]) #+(len(self.model[prev_tok].keys())*alpha))
        totalprob=sum(self.model[prev_tok].values())-self.model[prev_tok][self.tok_counter]
        for (prev_tok, dct) in self.model.items():
            for (curr_tok, val) in self.model[prev_tok].items():
                if curr_tok != self.tok_counter: # to avoid normalizing the counter token
                    self.model[prev_tok][curr_tok] /= totalprob
        print("Done with modeling building\nVocab size = {}".format(len(self.vocab)))

Run the following code to make sure your implementation is executable.

In [4]:
bigram_dis = BigramLM_Discounted()
bigram_dis.build("arxiv/trn-arxiv.txt")
bigram_dis.generate()

Done with modeling building
Vocab size = 12455


'<start> lead2gold searches gradually increasing pressure but are evaluated our model that reported to produce reasonable accuracy of domains such formulation'

For the purpose of comparison, let's also build a language model without discounting. 

In [5]:
bigram = BigramLM()
bigram.build("arxiv/trn-arxiv.txt")
bigram.generate()

Done with modeling building
Vocab size = 12455


'<start> the text generation time <end>'

### 1.2 Edge Cases (2 points)

Please design a edge case to demonstrate the value of the discounting techniques. Each edge case will be a sentence, which can demonstrate the difference between a language model with and without discounting. 

The requirement of forming an edge case

- it needs to be an English word sequence
- there is a significant difference between log probabilities given by the model with and without discounting. 

In [6]:
def findedgecase(text):
    with open(text) as file:
        probnorm = 1.0
        probdis = 1.0
        maxval = 0.0
        maxsent = ""
        nextline = file.readline()
        for i in range(0, 63259, 1): 
            line = ""
            try: 
                line = nextline
                probnorm = bigram.logprob(line)
                probdis = bigram_dis.logprob(line)
                #print(probnorm,probdis)
                #print(line)
                nextline = file.readline()
                if abs(probnorm-probdis)>maxval:
                    maxsent = line
                    maxval = abs(probnorm-probdis)
            except ZeroDivisionError:
                probnorm = 1.0
                probdis = 1.0
                nextline = file.readline()
        print(maxval)
        print(maxsent)

In [7]:
textsent = findedgecase("arxiv/trn-arxiv.txt")

1.6965051163887779
<start> simple review by linguist citing many articles by physicists quantitative methods agent based computer simulations language dynamics language typology historical linguistics <end>



In [8]:
# =====================================
# TODO: replace the following None with an edge case that you defined
text = "<start> simple review by linguist citing many articles by physicists quantitative methods agent based computer simulations language dynamics language typology historical linguistics <end>"
# =====================================
logprob_bigram = bigram.logprob(text)
logprob_bigram_dis = bigram_dis.logprob(text)
print("Text = {}\n Log prob on the original model {:.4f}\n Log prob on the model with discounting {}".format(text, logprob_bigram, logprob_bigram_dis))

Text = <start> simple review by linguist citing many articles by physicists quantitative methods agent based computer simulations language dynamics language typology historical linguistics <end>
 Log prob on the original model -166.6402
 Log prob on the model with discounting -164.9436915775177


## 2 Neural Language Models

Define a tokenization function for language modeling. Particularly, this tokenization function not only converts texts into a list of tokens, but also handle the padding operation, which is critical for mini-batch training in neural network modeling. 

In [9]:
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer

tokenizer = Tokenizer(WordLevel(unk_token="UNK"))
tokenizer.add_special_tokens(['<pad>', 'UNK'])
tokenizer.pre_tokenizer = Whitespace()
trainer = WordLevelTrainer()
tokenizer.train(['arxiv/trn-arxiv.txt'], trainer)
tokenizer.enable_padding(pad_id=tokenizer.token_to_id('<pad>'), pad_token='<pad>')

In [10]:
output = tokenizer.encode("Hello, y'all! How are you ?")
print(output.ids)
print(tokenizer.token_to_id('<pad>'))
print(tokenizer.get_vocab_size())

[13, 13, 13, 1628, 137, 13, 13, 20, 4623, 13]
0
12376


Create the dataset iterators for training and evaluation. The following code will convert each set (training or development) into a list of mini batches. You can change the mini-batch size with the parameter `batch_size` in the `DataLoader` function.

In [11]:
from datasets import load_dataset
import torch.utils
dataset = load_dataset('text', data_files={'train': 'arxiv/trn-arxiv.txt', 'dev': 'arxiv/dev-arxiv.txt'})

def collate(examples):
    texts = [ex['text'] for ex in examples]
    return torch.LongTensor([text.ids for text in tokenizer.encode_batch(texts)])

train_iter = torch.utils.data.DataLoader(dataset["train"], batch_size=32, collate_fn=collate)
dev_iter = torch.utils.data.DataLoader(dataset["dev"], batch_size=32, collate_fn=collate)

Using custom data configuration default-87fef769e91ff912
Reusing dataset text (/Users/sophiawalton/.cache/huggingface/datasets/text/default-87fef769e91ff912/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


  0%|          | 0/2 [00:00<?, ?it/s]

The following code will do two things for you:

- Load the necessary packages for neural network modeling from PyTorch
- Define the random seeds: by default Python and PyTorch will use a different random seed each time you run the code. Creating some randomness is good in general, but really not helpful when you debug the code. So, having a pre-defined random seed will make sure your code starts from the exactly same point everytime, which will make it relatively easier to identify any potential bugs. For example, the following code should print exactly the following tensor every time you run it

> tensor([-2.0157,  2.0106,  0.0583,  0.0656, -1.6534])

In [12]:
import torch
from torch import optim
from torch.nn.utils import clip_grad_norm_ as clip_grad_norm
import torch.nn as nn

## Random seeds, to make the results reproducible
seed = 49
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(torch.randn(5))

tensor([-2.0157,  2.0106,  0.0583,  0.0656, -1.6534])


### 2.1 Neural Network Implementation (4 points)

In the following class `SimpleRNN`, you need to complete two parts of the code

- In the **initialization function**, define the following three components. For each component, make sure you get the dimensionality right
 - [Word embedding layer](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
 - [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) as the nonlinear transition function
 - [a fully connected layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) for computing the logit
- In the **forward function**, define the LSTM language model that can read a mini-batch of texts and compute the logits of word predictions
 - In the definition of a softmax classifier
 $$p(x_{t+1}\mid x_{1:t}) \approx \exp(U\cdot h_{t})$$
 `logit` is defined as $U\cdot h_{t}$

In [13]:
import numpy as np
import magnum

#import long
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, input_dim=16, hidden_dim=16, num_layers=1, pad_idx=None):
        ''' Initialization function
        
        vocab_size - vocab size
        input_dim - word embedding dimension (w)
        hidden_dim - dimension of hidden states (h)
        num_layers - number of LSTM layers (> 2: stacked LSTM)
        pad_idx - padding token index
        '''
        super(SimpleRNN, self).__init__()
        self.nn_layers = nn.ModuleList()
        self.vocab_size = vocab_size # Vocab size
        self.embed_size = input_dim # Word embedding dimension
        self.hidden_dim = hidden_dim # Hidden dimension
        self.num_layers = num_layers # Number of layers
        #word embedding layer - padding = none
        self.embed = nn.Embedding(self.vocab_size, self.embed_size,padding_idx=pad_idx)
        #LSTM model
        self.lstm = nn.LSTM(input_size=self.embed_size,hidden_size=self.hidden_dim,num_layers=self.num_layers,batch_first=True)
        #fully connected layer
        self.fc = nn.Linear(self.hidden_dim, self.vocab_size)
      
    def forward(self, input):
        embeds = self.embed(input) # B x T x E
        batch_size = input.size(0)
        # ======================================
        # TODO: complete the following code for defining a RNN LM
        lstm_out, *_ = self.lstm(embeds)
        logits = self.fc(lstm_out) 
        # =======================================
        return logits #dims: batch size, sequence length, vocab size
        

### 2.2 Training with Mini-batch (2 points)

Please complete the following code by defining the loss function. The loss function has been defined with the [cross entropy function](), as shown in the following code 
```
loss_func = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id('<pad>'), reduce=True)
```
`ignore_index` is to make sure that the overall loss does not include the predictions on padding tokens, `reduce=True` means the returned loss from this function is the average NLL. 

To complete the code, you need to specify the two inputs of the loss function: `predicted` and `target`

- `predicted`: the logits of the input words
- `target`: the word indices of output words

In [14]:
loss_func = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id('<pad>'), reduce=True)

def batch_train(batch, model, optimizer, loss_func=loss_func, grad_clip=5.0):
    '''Training with one mini-batch

    batch - one mini-batch data
    model - an instance of RNN LM
    optimizer - the optimization function (e.g., SGD or Adam)
    loss_func - the pre-defined loss function
    grad_clip - hyper-parameter of grad clipping
    '''
    # Forward 
    model.train()
    logits = model(batch)

    # ============================================
    # TODO: Define predicted and target for the following loss function
    predicted = logits.transpose(1,2) # dims: batch size, vocab size, sequence length
    target = batch # dims: batch size, sequence length
    # ============================================
    loss = loss_func(predicted, target)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    clip_grad_norm(model.parameters(), grad_clip) # Norm  clipping
    optimizer.step()
    
    return loss.item()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


### 2.3 Perplexity (3 points)

For a given evaluation dataset $\{\bf{x}_i\}_{i=1}^{N}$, perplexity is defined as 
$$\text{Perplexity} = \exp \big(-\frac{1}{T}\sum_{i=1}^{N}\sum_{t=1}\log p(x_{i,t}\mid x_{i,1:t-1})\big)$$
where $T$ is the total number of predictions that a language model makes on the evaluation data (as we explained in the lecture), $N$ is the total number of sentences in the evaluation dataset.

In the following code, please implement the function that can read all the mini-batches in the deveopment set for evaluating the model performance during training. As you can find out from the next code block, this function will be used during training to measure the model performance after certain training iterations. 

This function needs to read all the mini-batch from the validation set, which are all in the `dev_iter` as defined before. The implementation of this function will be mostly similar to the `batch_train` function in the last code block, but without backward steps. 

Unlike training, to calculate the perplexity, you will need to set the `reduce` parameter in the cross entropy function to be `False`. 

In addition, make sure your calculation of perplexity does not include the predictions on padding tokens. 

In [15]:
loss_func_2 = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id('<pad>'), reduce=False)

def perplexity(dev_iter, model, loss_func=loss_func_2):
    '''
    '''
    model.eval() # Set in the evaluation mode
    pplx = 0.0
    logs = 0
    for (b, batch) in enumerate(dev_iter):
        logits = model(batch)
        loss = loss_func(logits.transpose(1,2),batch)
        pplx += loss.sum()
        logs += torch.numel(loss)
        
    pplx = torch.exp(pplx/logs)
    return pplx

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


### 2.4 Combine All Components for Training (2 points)

Once you implement all the components, please run the following code and plot the perplexity curve on the development set over training. 

Based on our implementation, the training procedure needs to run at least five epochs to get a good result, which will take about 10 - 30 minutes on CPUs to finish. Depending the computational resources you have, it's optional to increase the number of training epochs to get even better results.

In [ ]:
model = SimpleRNN(tokenizer.get_vocab_size(), input_dim=16, hidden_dim=16,
                  num_layers=1, pad_idx=tokenizer.token_to_id('<pad>'))
optimizer = optim.Adam(model.parameters())

from tqdm.notebook import trange, tqdm
eval_step = 200
epochs = 5
pplx_lst = []
for epoch in trange(epochs):
    for b, batch in enumerate(train_iter):
        loss = batch_train(batch, model, optimizer)
        if b % eval_step == 0:
            pplx = perplexity(dev_iter, model)
            pplx_lst.append((b,pplx))
            print(f"Epoch: {epoch}, Batch: {b}, Batch Loss: {loss}, Perplexity: {pplx}", flush=True)
                        
# ===============================================
# TODO: plot the perplexity curve on the development set.


# ================================================

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0, Batch: 0, Batch Loss: 9.412986755371094, Perplexity: 122.95208740234375


In [ ]:
import matplotlib as plt
bs = [b for b,p in pplx_lst]
pplxs = [p for b,p in pplx_lst]
plt.plot(bs,pplxs)
plt.plot(bs,pplxs,'or')
plt.show()
# ===========